In [1]:
#Libraries
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Matplotlib is building the font cache; this may take a moment.


In [ ]:
# Preprocess original dataset

# df = pd.read_csv('data/final_animedataset.csv')
# print(f"Shape: {df.shape}")
# df.head()

Shape: (35305695, 13)


,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre
0,karthiga,21,9,2255153,Female,One Piece,TV,Manga,8.54,423868,91.0,35,"Action, Adventure, Comedy, Super Power, Drama,..."
1,karthiga,59,7,2255153,Female,Chobits,TV,Manga,7.53,175388,1546.0,188,"Sci-Fi, Comedy, Drama, Romance, Ecchi, Seinen"
2,karthiga,74,7,2255153,Female,Gakuen Alice,TV,Manga,7.77,33244,941.0,1291,"Comedy, School, Shoujo, Super Power"
3,karthiga,120,7,2255153,Female,Fruits Basket,TV,Manga,7.77,167968,939.0,222,"Slice of Life, Comedy, Drama, Romance, Fantasy..."
4,karthiga,178,7,2255153,Female,Ultra Maniac,TV,Manga,7.26,9663,2594.0,2490,"Magic, Comedy, Romance, School, Shoujo"


In [ ]:
# # Drop Irrelevant Rows (type: movie, other) \
# df = df[(df['type'] == 'TV')]
# df.value_counts('type')

type
TV    23703867
Name: count, dtype: int64

In [ ]:
# # Drop rows with empty username
# df.dropna(subset = ['username'], inplace=True)

# # Check for duplicate rows/null values
# null = df.isnull().sum()
# dupe = df.duplicated().sum()

# print(f"Number of Null Values: {null} \n Number of Duplicate Rows: {dupe}")

Number of Null Values: username      0
anime_id      0
my_score      0
user_id       0
gender        0
title         0
type          0
source        0
score         0
scored_by     0
rank          0
popularity    0
genre         0
dtype: int64 
 Number of Duplicate Rows: 0


In [13]:
df = pd.read_csv('data/anime_tv_dataset.csv.zip')

In [14]:
unique_user = df['username'].nunique()
unique_anime = df['anime_id'].nunique()
num_ratings = len(df)
print(f"Total Number of Unique Users: {unique_user}")
print(f"Total Number of Unique Anime Titles: {unique_anime}")
print(f"Total Number of User Ratings: {num_ratings}")


Total Number of Unique Users: 115859
Total Number of Unique Anime Titles: 2973
Total Number of User Ratings: 23703692


In [7]:
# What is the distribution of user ratings (my_score) across all anime?

# sns.histplot(data=df, x='my_score', bins=10)
# plt.title('Distribution of User Ratings')
# plt.xlabel('User Rating (my_score)')
# plt.ylabel('Count')
# plt.show()

In [8]:
# Average User Scores Across Different Anime Genres
# Explode the Genre (Convert the list of genres to its own row)
# Keep this uncommented if you dont want your computer to explode
# df_reduced = df[['genre', 'my_score']].copy()

# # Split and explode 'genre'
# df_reduced['genre'] = df_reduced['genre'].str.split(', ')
# df_exploded = df_reduced.explode('genre')

# # Group by 'genre' and calculate the mean
# genre_avg_scores = df_exploded.groupby('genre')['my_score'].mean().reset_index()

# # Plot the results
# plt.figure(figsize=(12, 6))
# sns.barplot(data=genre_avg_scores, x='genre', y='my_score')
# plt.title('Average User Score by Genre')
# plt.xlabel('Genre')
# plt.ylabel('Average User Score')
# plt.xticks(rotation=90)
# plt.tight_layout()
# plt.show()

In [9]:
# Do popular animes have higher scores (Bad Anime, Popular)

# sns.scatterplot(data=df, x='popularity', y='score')
# plt.title('Anime Score vs. Popularity')
# plt.xlabel('Popularity Rank')
# plt.ylabel('Anime Score')
# plt.gca().invert_xaxis()  # Invert x-axis if lower rank means higher popularity
# plt.show()

In [10]:
# Do highly rated animes have more users rate them?

# sns.scatterplot(data=df, x='rank', y='scored_by')
# plt.title('Number of Users Scoring vs. Anime Rank')
# plt.xlabel('Anime Rank')
# plt.ylabel('Number of Users Who Scored')
# plt.gca().invert_xaxis()  # Invert x-axis if lower rank is better
# plt.show()

In [11]:
# Get descriptive statistics
# print(df['my_score'].describe())

In [12]:
# Reduce memory usage by selecting only the 'genre' column
# df_genre = df[['genre']].copy()

# # Split and explode the 'genre' column
# df_genre['genre'] = df_genre['genre'].str.split(', ')
# df_exploded = df_genre.explode('genre')

# # Count the genres
# genre_counts = df_exploded['genre'].value_counts().reset_index()
# genre_counts.columns = ['genre', 'count']

# # Optimize data types
# genre_counts['count'] = genre_counts['count'].astype('int32')

# # Sort genres by count
# genre_counts = genre_counts.sort_values(by='count', ascending=False)

# # Select a bright color palette
# num_genres = genre_counts['genre'].nunique()
# palette_colors = sns.color_palette('bright', n_colors=num_genres)

# # Map colors to genres
# genre_color_mapping = dict(zip(genre_counts['genre'], palette_colors))

# # Plot the genre popularity with bright colors
# plt.figure(figsize=(12, 6))
# sns.barplot(
#     data=genre_counts,
#     x='genre',
#     y='count',
#     hue='genre',       # Assign hue to 'genre' to map colors correctly
#     dodge=False,       # Ensure bars are not dodged
#     palette=genre_color_mapping,
#     legend=False       # Hide legend if not needed
# )
# plt.title('Popularity of Genres')
# plt.xlabel('Genre')
# plt.ylabel('Count')
# plt.xticks(rotation=90)
# plt.tight_layout()
# plt.show()


In [13]:
# # Select necessary columns
# df_genre = df[['genre', 'my_score', 'gender']].copy()

# # Filter out rows with missing or zero 'my_score' values
# df_genre = df_genre[df_genre['my_score'] > 0]

# # Optimize data types to reduce memory usage
# df_genre['my_score'] = df_genre['my_score'].astype('float32')
# df_genre['gender'] = df_genre['gender'].astype('category')

# # Split the 'genre' column
# df_genre['genre'] = df_genre['genre'].str.split(', ')

# # Explode the 'genre' column
# df_exploded = df_genre.explode('genre')

# # Plot box plots
# plt.figure(figsize=(24, 8))  # Increase figure size
# sns.boxplot(
#     data=df_exploded,
#     x='genre',
#     y='my_score',
#     hue='gender',
#     width=0.8,       # Adjust the width of the boxes
#     fliersize=2      # Adjust the size of outlier markers
# )
# plt.title('User Ratings by Genres and Gender', fontsize=16)
# plt.xlabel('Genre', fontsize=14)
# plt.ylabel('User Rating (my_score)', fontsize=14)
# plt.xticks(rotation=90, fontsize=12)  # Rotate labels, align right, adjust font size
# plt.yticks(fontsize=12)
# plt.legend(title='Gender', fontsize=12, title_fontsize=14)
# plt.tight_layout()
# plt.show()

In [14]:
df.head()

,username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre
0,karthiga,21,9,2255153,Female,One Piece,TV,Manga,8.54,423868,91.0,35,"Action, Adventure, Comedy, Super Power, Drama,..."
1,karthiga,59,7,2255153,Female,Chobits,TV,Manga,7.53,175388,1546.0,188,"Sci-Fi, Comedy, Drama, Romance, Ecchi, Seinen"
2,karthiga,74,7,2255153,Female,Gakuen Alice,TV,Manga,7.77,33244,941.0,1291,"Comedy, School, Shoujo, Super Power"
3,karthiga,120,7,2255153,Female,Fruits Basket,TV,Manga,7.77,167968,939.0,222,"Slice of Life, Comedy, Drama, Romance, Fantasy..."
4,karthiga,178,7,2255153,Female,Ultra Maniac,TV,Manga,7.26,9663,2594.0,2490,"Magic, Comedy, Romance, School, Shoujo"


In [ ]:
# Data Cleaning
# Drop rows with missing values in essential columns
data = df.dropna(subset=['my_score', 'gender', 'type', 'source', 'genre'])

# Feature Selection
features = ['gender', 'type', 'source', 'score', 'rank', 'popularity', 'genre']
target = 'my_score'

X = data[features]
y = data[target]

# Feature Encoding
categorical_features = ['gender', 'type', 'source', 'genre']

# Apply OneHotEncoding 
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
    remainder='passthrough')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")

# Save the pipeline to a file
with open('trained_pipeline_linear_reg.pkl', 'wb') as file:
    pickle.dump(pipeline, file)
print("Model saved to 'trained_pipeline_linear_reg.pkl'.")

# Recommender
def recommend_anime(user_id, num_recommendations=5):
    # Get the user's gender
    user_data = data[data['user_id'] == user_id]
    if user_data.empty:
        print(f"No data found for user_id: {user_id}")
        return pd.DataFrame()
    
    user_gender = user_data['gender'].iloc[0]
    
    # Get all anime rated by user
    rated_anime = user_data['anime_id'].unique()
    
    # Get all unique anime not yet rated by user
    unrated_anime = data[~data['anime_id'].isin(rated_anime)].drop_duplicates(subset=['anime_id'])
    
    # Check if there any unrated anime
    if unrated_anime.empty:
        print("No unrated anime available for recommendation.")
        return pd.DataFrame()
    
    # 'gender' is a user attribute, set it to the user's gender for all
    X_unrated = unrated_anime[features].copy()
    X_unrated['gender'] = user_gender  # Set user's gender
    
    # Predict ratings using the pipeline
    predicted_scores = pipeline.predict(X_unrated)
    
    # Add predictions to the dataframe
    unrated_anime = unrated_anime.copy()
    unrated_anime['predicted_score'] = predicted_scores
    
    # Sort by predicted_score in descending order and select top N
    recommendations = unrated_anime.sort_values(
        by='predicted_score', ascending=False).head(num_recommendations)
    
    return recommendations[['title', 'predicted_score']]

# Example recommendation for a user
user_id_example = 2255153  # Replace with an actual user_id from your dataset
recommendations = recommend_anime(user_id_example)
print("Recommended Anime:")
print(recommendations)


Mean Squared Error: 14.62
Model saved to 'trained_pipeline_linear_reg.pkl'.
Recommended Anime:
                                   title  predicted_score
177                           Death Note         7.641494
202   Code Geass: Hangyaku no Lelouch R2         7.208718
179      Code Geass: Hangyaku no Lelouch         6.751835
466                        One Punch Man         6.733394
1384                        Angel Beats!         6.570280


In [16]:
# Extract the Linear Regression model from the pipeline
linear_model = pipeline.named_steps['regressor']

# Get all feature names dynamically after preprocessing
all_feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()

# Get the coefficients from the linear regression model
coefficients = linear_model.coef_

# Combine feature names and coefficients into a DataFrame
feature_impact = pd.DataFrame({
    'Feature': all_feature_names,
    'Coefficient': coefficients
})

# Add an 'Impact' column as the absolute value of the coefficients
feature_impact['Impact'] = feature_impact['Coefficient'].abs()

# Sort features by impact in descending order
sorted_feature_impact = feature_impact.sort_values(by='Impact', ascending=False)

# Display the top features with the most impact
print("Top Features with the Most Impact on Score:")
print(sorted_feature_impact.head(10))  # Show top 10 impactful features


Top Features with the Most Impact on Score:
                                                Feature  Coefficient    Impact
1675  cat__genre_Mystery, Police, Psychological, Sup...     2.642213  2.642213
1811                        cat__genre_Sci-Fi, Thriller    -2.620359  2.620359
561   cat__genre_Action, Military, Sci-Fi, Super Pow...     2.226861  2.226861
113   cat__genre_Action, Adventure, Fantasy, Game, R...     2.088396  2.088396
232   cat__genre_Action, Comedy, Drama, School, Supe...     2.053278  2.053278
562   cat__genre_Action, Military, Sci-Fi, Super Pow...     1.885727  1.885727
30    cat__genre_Action, Adventure, Comedy, Drama, F...     1.757219  1.757219
1671  cat__genre_Mystery, Horror, Supernatural, Thri...     1.750231  1.750231
647   cat__genre_Action, Sci-Fi, Comedy, Parody, Sup...     1.717550  1.717550
475   cat__genre_Action, Horror, Psychological, Supe...     1.715674  1.715674


In [17]:
# Data Cleaning
# Drop rows with missing values in essential columns
data = df.dropna(subset=['my_score', 'gender', 'source', 'genre'])

# Feature Selection
features = ['gender', 'source', 'score', 'genre']
target = 'my_score'

X = data[features]
y = data[target]

# List of categorical features
categorical_features = ['gender', 'source', 'genre']

# Apply OneHotEncoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
    remainder='passthrough')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")

# Save the pipeline to a file
with open('trained_pipeline_linear_reg.pkl', 'wb') as file:
    pickle.dump(pipeline, file)
print("Model saved to 'trained_pipeline_linear_reg.pkl'.")

# Recommendations
def recommend_anime(user_id, num_recommendations=5):
    # Get the user's gender
    user_data = data[data['user_id'] == user_id]
    if user_data.empty:
        print(f"No data found for user_id: {user_id}")
        return pd.DataFrame()
    
    user_gender = user_data['gender'].iloc[0]
    
    # Get all anime rated by the user
    rated_anime = user_data['anime_id'].unique()
    
    # Get all unique anime not yet rated by the user
    unrated_anime = data[~data['anime_id'].isin(rated_anime)].drop_duplicates(subset=['anime_id'])
    
    # Check if there are any unrated anime
    if unrated_anime.empty:
        print("No unrated anime available for recommendation.")
        return pd.DataFrame()
    
    # Prepare features for prediction
    # Since 'gender' is a user attribute, set it to the user's gender for all
    X_unrated = unrated_anime[features].copy()
    X_unrated['gender'] = user_gender  # Set user's gender
    
    # Predict ratings using the pipeline
    predicted_scores = pipeline.predict(X_unrated)
    
    # Add predictions to the dataframe
    unrated_anime = unrated_anime.copy()
    unrated_anime['predicted_score'] = predicted_scores
    
    # Sort by predicted_score in descending order and select top N
    recommendations = unrated_anime.sort_values(
        by='predicted_score', ascending=False).head(num_recommendations)
    
    return recommendations[['title', 'predicted_score']]

# Example recommendation for a user
user_id_example = 2255153  # Replace with an actual user_id from your dataset
recommendations = recommend_anime(user_id_example)
print("Recommended Anime:")
print(recommendations)


Mean Squared Error: 14.51
Model saved to 'trained_pipeline_linear_reg.pkl'.
Recommended Anime:
                                  title  predicted_score
177                          Death Note         6.987652
202  Code Geass: Hangyaku no Lelouch R2         6.882797
466                       One Punch Man         6.651447
230    Fullmetal Alchemist: Brotherhood         6.432273
179     Code Geass: Hangyaku no Lelouch         6.414187


In [18]:
# Median Baseline
med_score = y_train.median()
y_pred_med = np.full_like(y_test, med_score)
mse_med = mean_squared_error(y_test, y_pred_med)
print(f"Median Baseline MSE: {mse_med:.2f}")

Median Baseline MSE: 17.60
